<a href="https://colab.research.google.com/github/tap-o-chek/Multiple-processing-course/blob/main/labs/MP_Lesson_2_%D0%96%D0%B8%D0%B4%D0%B5%D0%BB%D0%B5%D0%B2%D0%B0_%D0%A1%D0%BE%D1%84%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Упражнение 1.** Запустите код. Попробуйте объяснить, почему LIST - пуст.

In [ ]:
import multiprocessing

def worker():
    LIST.append('item')

LIST = []

if __name__ == "__main__":
    processes = [
        multiprocessing.Process(target=worker)
        for _ in range(5)
    ]
    for p in processes:
        p.start()
    for p in processes:
        p.join()
    print(LIST)

[]


Объяснение: потому что процессы мультипроцессинга работают каждый в своей ячейке памяти, поэтому и добавление элементов в лист идёт в различных ячейках, а так как мы выводим не результат выполнения конкретного потока, а обычный лист, то он выводится пустым, ибо с ним по факту и не взаимодействовали

# **Упражнение 2.** Запустите код. Какая проблема здесь возникает? Исправьте её.

In [ ]:
import threading
import time

l1 = threading.Lock()
l2 = threading.Lock()

def f1(name):
    print('thread',name,'about to lock l1')
    with l1:
        print('thread',name,'has lock l1')
        time.sleep(0.3)
        print('thread',name,'about to lock l2')
        with l2:
            print('thread',name)

def f2(name):
    print('thread',name,'about to lock l2')
    with l1:
        print('thread',name,'has lock l2')
        time.sleep(0.3)
        print('thread',name,'about to lock l1')
        with l2:
            print('thread',name)

if __name__ == '__main__':
    t1=threading.Thread(target=f1, args=['t1',])
    t2=threading.Thread(target=f2, args=['t2',])

    t1.start()
    t2.start()

    t1.join()
    t2.join()

thread t1 about to lock l1
thread t1 has lock l1
thread t2 about to lock l2
thread t1 about to lock l2
thread t1
thread t2 has lock l2
thread t2 about to lock l1
thread t2


Проблема потому что каждая функция пытается захватить оба замка в разном порядке, что приводит к возможности взаимной блокировки , когда одна функция ждет освобождения замка, удерживаемого другой функцией, а вторая функция ждет освобождения замка, удерживаемого первой функцией. Можно изменить порядок, в котором замки захватываются в функциях так, чтобы обе функции захватывали замки в одном и том же порядке.

In [ ]:
import threading
import time

l1 = threading.Lock()
l2 = threading.Lock()

def f1(name):
    print('thread',name,'about to lock l1')
    with l1:
        print('thread',name,'has lock l1')
        time.sleep(0.3)
        print('thread',name,'about to lock l2')
    with l2:
        print('thread',name)

def f2(name):
    print('thread',name,'about to lock l2')
    with l2:
        print('thread',name,'has lock l2')
        time.sleep(0.3)
        print('thread',name,'about to lock l1')
    with l1:
        print('thread',name)

if __name__ == '__main__':
    t1=threading.Thread(target=f1, args=['t1',])
    t2=threading.Thread(target=f2, args=['t2',])

    t1.start()
    t2.start()

    t1.join()
    t2.join()


thread t1 about to lock l1
thread t1 has lock l1
thread t2 about to lock l2
thread t2 has lock l2
thread t1 about to lock l2
thread t2 about to lock l1
thread t2
thread t1


#**Упражнение 3.**

**Упражнение 3.** Вам необходимо вычислить значение функции f = x**2 + x * 2 + 10 * x для различных значений аргументов (10 случайных чисел).

1. Сделайте программу без использования потоков/процессов и измерьте время.

2. Разбейте задачу на несколько потоков (отдельный поток для каждого аргумента и отдельный поток для каждого слагаемоего функции). Запустите программу. Измерьте время.

3. Разбейте задачу на несколько процессов (отдельный процесс для каждого аргумента и отдельный процесс для каждого слагаемоего функции). Запустите программу. Измерьте время.

Для синхронизации можно использовать барьеры.

Объясните полученные результаты.

In [ ]:
#Без потоков
import time
import random

def f(x):
    return x**2 + x * 2 + 10 * x

def main():
    start_time = time.time()
    values = [random.randint(1, 100) for _ in range(10)]
    results = [f(x) for x in values]
    end_time = time.time()
    print("Results without concurrency:", results)
    print("Time taken without concurrency:", end_time - start_time, "seconds")

if __name__ == "__main__":
    main()

Results without concurrency: [8064, 10780, 1485, 8245, 9373, 1408, 1989, 220, 253, 4453]
Time taken without concurrency: 7.128715515136719e-05 seconds


Время выполнения будет увеличиваться с увеличением числа аргументов

In [ ]:
#Несколько потоков
import time
import random
from threading import Thread, Barrier

def f(x, barrier, result):
    result.append(x**2 + x * 2 + 10 * x)
    barrier.wait()

def main():
    start_time = time.time()
    values = [random.randint(1, 100) for _ in range(10)]
    barrier = Barrier(len(values))
    result = []

    threads = [Thread(target=f, args=(x, barrier, result)) for x in values]
    for t in threads:
        t.start()
    for t in threads:
        t.join()

    end_time = time.time()
    print("Results with threads:", result)
    print("Time taken with threads:", end_time - start_time, "seconds")

if __name__ == "__main__":
    main()

Results with threads: [7020, 5005, 10165, 448, 9568, 9765, 9373, 1813, 3808, 4320]
Time taken with threads: 0.010246992111206055 seconds


Так как поток выполняется за раз, то они не дают особого преимущества в производительности

In [ ]:
#Несколько процессов
import time
import random
from multiprocessing import Process, Barrier

def f(x, barrier, result):
    result.append(x**2 + x * 2 + 10 * x)
    barrier.wait()

if __name__ == "__main__":
    start_time = time.time()
    values = [random.randint(1, 100) for _ in range(10)]
    barrier = Barrier(len(values))
    result = []

    processes = [Process(target=f, args=(x, barrier, result)) for x in values]
    for p in processes:
        p.start()
    for p in processes:
        p.join()

    end_time = time.time()
    print("Results with processes:", result)
    print("Time taken with processes:", end_time - start_time, "seconds")

Results with processes: []
Time taken with processes: 0.1182703971862793 seconds


так как процессы используют несколько ядер, что сказывается на производительности

# Упражнение 4.

**Упражнение 4.** Смоделируйте следующую ситуацию с использованием семафоров (и затем событий). То есть нужно два варианта: с семафорами и с событиями.

1. Есть 5 касс и 20 покупателей. Все они хотят купить билеты на матч. На билетах не указаны места.

2. После покупки билета покупатель бежит на стадион (у каждого разная скорость бега, это намек, что надо сделать sleep) и занимает свободное место.

Программа должна вывести логированные события по типу:

client 0, service time (ticket): 1.0004174709320068

client 1, service time (ticket): 1.0005174709320068

client 1, runnig time to stad:   1.2004174709320068

client 0, runnig time to stad:   1.5004174709320068

In [ ]:
import threading
import time
import random

num_cashiers = 5
num_clients = 20
cashier_semaphore = threading.Semaphore(num_cashiers)

def buy_ticket(client_id):
    global cashier_semaphore
    cashier_semaphore.acquire()
    print(f"client {client_id}, service time (ticket): {time.time()}")
    time.sleep(random.uniform(0.5, 1.5))  # время обслуживания клиента
    cashier_semaphore.release()

def go_to_stadium(client_id):
    print(f"client {client_id}, running time to stad: {time.time()}")

def client_action(client_id):
    buy_ticket(client_id)
    go_to_stadium(client_id)

if __name__ == "__main__":
    threads = []
    for i in range(num_clients):
        t = threading.Thread(target=client_action, args=(i,))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()


client 0, service time (ticket): 1709837487.6926653
client 1, service time (ticket): 1709837487.69449
client 2, service time (ticket): 1709837487.6955638
client 3, service time (ticket): 1709837487.6969833
client 4, service time (ticket): 1709837487.6978722
client 4, running time to stad: 1709837488.5118492
client 5, service time (ticket): 1709837488.5128956
client 3, running time to stad: 1709837488.7314634
client 6, service time (ticket): 1709837488.731825
client 1, running time to stad: 1709837489.0082078client 7, service time (ticket): 1709837489.0083094

client 2, running time to stad: 1709837489.0445123client 8, service time (ticket): 1709837489.0446277

client 0, running time to stad: 1709837489.0925975client 9, service time (ticket): 1709837489.0927093

client 5, running time to stad: 1709837489.2342234
client 10, service time (ticket): 1709837489.2347093
client 6, running time to stad: 1709837489.716757
client 11, service time (ticket): 1709837489.7173088
client 10, running ti

In [ ]:
import threading
import time
import random

num_cashiers = 5
num_clients = 20
cashier_semaphore = threading.Semaphore(num_cashiers)

def buy_ticket(client_id):
    global cashier_semaphore
    cashier_semaphore.acquire()
    print(f"client {client_id}, service time (ticket): {time.time()}")
    time.sleep(random.uniform(0.5, 1.5))  # время обслуживания клиента
    cashier_semaphore.release()

def go_to_stadium(client_id):
    print(f"client {client_id}, running time to stad: {time.time()}")

def client_action(client_id):
    buy_ticket(client_id)
    go_to_stadium(client_id)

if __name__ == "__main__":
    threads = []
    for i in range(num_clients):
        t = threading.Thread(target=client_action, args=(i,))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()

client 0, service time (ticket): 1709838843.8163457client 1, service time (ticket): 1709838843.817714

client 2, service time (ticket): 1709838843.8194785
client 3, service time (ticket): 1709838843.8206139client 4, service time (ticket): 1709838843.8212564

client 2, running time to stad: 1709838844.5470347
client 5, service time (ticket): 1709838844.548684
client 0, running time to stad: 1709838844.654987
client 6, service time (ticket): 1709838844.6552556
client 3, running time to stad: 1709838845.0440392
client 7, service time (ticket): 1709838845.0444012
client 4, running time to stad: 1709838845.1420274
client 8, service time (ticket): 1709838845.142503
client 6, running time to stad: 1709838845.267558
client 9, service time (ticket): 1709838845.2680447
client 1, running time to stad: 1709838845.285031
client 10, service time (ticket): 1709838845.2853851
client 8, running time to stad: 1709838845.647895client 11, service time (ticket): 1709838845.648031

client 5, running time to

In [ ]:
import threading
import time
import random

num_cashiers = 5
num_clients = 20
cashier_event = threading.Event()

def buy_ticket(client_id):
    global cashier_event
    cashier_event.wait()
    print(f"client {client_id}, service time (ticket): {time.time()}")
    time.sleep(random.uniform(0.5, 1.5))  # время обслуживания клиента
    cashier_event.set()

def go_to_stadium(client_id):
    print(f"client {client_id}, running time to stad: {time.time()}")

def client_action(client_id):
    buy_ticket(client_id)
    go_to_stadium(client_id)

if __name__ == "__main__":
    threads = []
    for i in range(num_clients):
        t = threading.Thread(target=client_action, args=(i,))
        threads.append(t)
        t.start()
        if (i+1) % num_cashiers == 0:
            cashier_event.clear()
    for t in threads:
        t.join()

# **Упражнение 5.** Покажите и опишите разничу между Semaphore и BoundedSemaphore.

In [ ]:
import threading
import time

def task(semaphore, thread_id):
    semaphore.acquire()
    print(f"Thread {thread_id} is waiting")
    print(f"Thread {thread_id} has acquired the semaphore")
    time.sleep(1)
    print(f"Thread {thread_id} is releasing the semaphore")
    semaphore.release()

if __name__ == "__main__":
    # Semaphore example
    print("Semaphore")
    semaphore = threading.Semaphore(2)  # Максимум два потока могут получить семафор одновременно
    threads = []
    for i in range(4):
        t = threading.Thread(target=task, args=(semaphore, i))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()

    # BoundedSemaphore example
    print("BoundedSemaphore")
    bounded_semaphore = threading.BoundedSemaphore(2)  # Максимум два потока могут получить семафор одновременно
    threads = []
    for i in range(4):
        t = threading.Thread(target=task, args=(bounded_semaphore, i))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()


Semaphore
Thread 0 is waiting
Thread 0 has acquired the semaphore
Thread 1 is waiting
Thread 1 has acquired the semaphore
Thread 0 is releasing the semaphore
Thread 2 is waiting
Thread 2 has acquired the semaphore
Thread 1 is releasing the semaphore
Thread 3 is waiting
Thread 3 has acquired the semaphore
Thread 2 is releasing the semaphore
Thread 3 is releasing the semaphore
BoundedSemaphore
Thread 0 is waiting
Thread 0 has acquired the semaphore
Thread 1 is waiting
Thread 1 has acquired the semaphore
Thread 0 is releasing the semaphore
Thread 2 is waiting
Thread 2 has acquired the semaphore
Thread 1 is releasing the semaphore
Thread 3 is waiting
Thread 3 has acquired the semaphore
Thread 2 is releasing the semaphore
Thread 3 is releasing the semaphore


Семафоры управляют атомарным счетчиком. Также метод .acquire() блокируется до тех пор, пока значения внутреннего счетчика не станет положительным. Если счетчик value не указан , значение по умолчанию равно 1.

BoundedSemaphore реализует ограниченные объекты семафоров. Ограниченный семафор проверяет, не превышает ли его текущее значение его начальное значение. Если это так, то возникает исключение.

В большинстве случаев семафоры используются для защиты ресурсов с ограниченной емкостью. Если семафор запускается слишком много раз, то это признак ошибки.